In [53]:
import requests
import pandas
import geopandas
app_token = "PoXtpm9UpT6UnvUOK8F0lp3Sp"

In [58]:
url_params ={"$$app_token": app_token,
            "$where":"(borocode='1' AND rw_type='1')",
            "$limit": "10000000"}

streets = geopandas.read_file(
    requests.get("https://data.cityofnewyork.us/resource/8rma-cm9c.geojson", 
    params=url_params).text).to_crs(epsg=2263)

streets

,rw_type,l_low_hn,pre_direct,st_width,r_zip,r_low_hn,bike_lane,post_type,r_blkfc_id,bike_trafd,...,snow_pri,borocode,trafdir,pre_modifi,st_label,physicalid,st_name,created,l_high_hn,geometry
0,1,50,None,42.0,10280,51,None,PL,1222601917,None,...,C,1,TW,None,BATTERY PL,3,BATTERY,2007-11-29,64,"MULTILINESTRING ((979278.645 196555.536, 97929..."
1,1,66,None,42.0,10280,65,None,PL,1222604562,None,...,C,1,TW,None,BATTERY PL,5,BATTERY,2007-11-29,82,"MULTILINESTRING ((979377.464 196797.796, 97950..."
2,1,84,None,42.0,10280,83,None,PL,1222600933,None,...,C,1,TW,None,BATTERY PL,6,BATTERY,2007-11-29,98,"MULTILINESTRING ((979503.340 197024.626, 97964..."
3,1,None,None,42.0,10280,None,None,PL,1222600931,None,...,C,1,TW,None,BATTERY PL,8,BATTERY,2007-11-29,None,"MULTILINESTRING ((979553.795 196059.672, 97952..."
4,1,None,None,24.0,10004,None,None,PL,212262395,None,...,C,1,TF,None,BATTERY PL,14,BATTERY,2011-04-22,None,"MULTILINESTRING ((980288.141 195963.026, 98026..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9342,1,None,None,28.0,10004,None,None,RD,1922602691,None,...,V,1,TW,None,FERRY LINE RD,197028,FERRY LINE,2016-01-22,None,"MULTILINESTRING ((980032.476 191764.998, 98002..."
9343,1,None,None,30.0,10028,None,None,None,1322601051,None,...,V,1,TW,None,DSNY MARINE TRANSFER STATION RP,197034,DSNY MARINE TRANSFER STATION RP,2007-11-29,None,"MULTILINESTRING ((999601.478 223130.813, 99979..."
9344,1,None,None,0.0,10128,None,None,None,0,None,...,None,1,TW,None,DSNY MARINE TRANSFER STATION RP,197037,DSNY MARINE TRANSFER STATION RP,2022-08-17,None,"MULTILINESTRING ((1000027.066 223065.336, 1000..."
9345,1,4,W,30.0,10023,1,None,ST,1322602331,None,...,S,1,FT,None,W 69 ST,197062,69,2007-11-29,48,"MULTILINESTRING ((990517.638 221369.065, 98999..."


In [55]:
url_params ={"$$app_token": app_token, 
            "$where":"(boro_nm='MANHATTAN')",
            "$limit": "10000000"}

incidents = geopandas.read_file(
    requests.get("https://data.cityofnewyork.us/resource/5uac-w243.geojson", 
    params=url_params).text).to_crs(epsg=2263)

incidents

,parks_nm,hadevelopt,lat_lon_zip,latitude,cmplnt_to_tm,y_coord_cd,susp_race,loc_of_occur_desc,patrol_boro,station_name,...,susp_sex,lat_lon_address,law_cat_cd,pd_cd,lat_lon_state,boro_nm,prem_typ_desc,vic_age_group,housing_psa,geometry
0,(null),(null),None,40.799522,21:50:00,230571,WHITE HISPANIC,INSIDE,PATROL BORO MAN NORTH,(null),...,F,None,VIOLATION,637,None,MANHATTAN,RESIDENCE - PUBLIC HOUSING,25-44,670,POINT (1002844.933 230571.541)
1,(null),(null),None,40.74828972,(null),211899,BLACK,(null),PATROL BORO MAN SOUTH,34 STREET,...,M,None,FELONY,273,None,MANHATTAN,TRANSIT - NYC SUBWAY,UNKNOWN,None,POINT (987526.003 211898.998)
2,UNION SQUARE PARK,(null),None,40.735698,(null),207311,UNKNOWN,(null),PATROL BORO MAN SOUTH,(null),...,M,None,VIOLATION,638,None,MANHATTAN,PARK/PLAYGROUND,25-44,None,POINT (987114.440 207311.382)
3,(null),(null),None,40.720761,12:40:00,201869,UNKNOWN,OPPOSITE OF,PATROL BORO MAN SOUTH,(null),...,U,None,FELONY,441,None,MANHATTAN,STREET,25-44,None,POINT (989658.896 201869.801)
4,(null),(null),None,40.770157,17:06:00,219866,WHITE,INSIDE,PATROL BORO MAN NORTH,(null),...,M,None,MISDEMEANOR,333,None,MANHATTAN,CHAIN STORE,UNKNOWN,None,POINT (989200.071 219866.247)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65352,(null),(null),None,40.73975138,04:01:00,208788,BLACK,(null),PATROL BORO MAN SOUTH,14 STREET,...,M,None,MISDEMEANOR,101,None,MANHATTAN,TRANSIT - NYC SUBWAY,25-44,None,POINT (983551.001 208787.997)
65353,(null),(null),None,40.729942,(null),205214,BLACK,INSIDE,PATROL BORO MAN SOUTH,(null),...,U,None,MISDEMEANOR,333,None,MANHATTAN,CHAIN STORE,UNKNOWN,None,POINT (990020.394 205214.807)
65354,(null),(null),None,40.8009303727402,17:15:00,231080,WHITE HISPANIC,(null),PATROL BORO MAN NORTH,(null),...,M,None,MISDEMEANOR,175,None,MANHATTAN,STREET,25-44,None,POINT (1000557.601 231083.015)
65355,(null),(null),None,40.838991,22:25:00,244950,BLACK,FRONT OF,PATROL BORO MAN NORTH,(null),...,M,None,MISDEMEANOR,113,None,MANHATTAN,RESIDENCE - APT. HOUSE,45-64,None,POINT (1001528.148 244950.553)


In [57]:
url_params ={"$$app_token": app_token, 
            "$where":"(arrest_boro='M')",
            "$limit": "10000000"}

arrests = geopandas.read_file(
    requests.get("https://data.cityofnewyork.us/resource/uip8-fykc.geojson", 
    params=url_params).text).to_crs(epsg=2263)

arrests

,perp_sex,latitude,arrest_precinct,y_coord_cd,jurisdiction_code,x_coord_cd,age_group,ky_cd,law_code,arrest_boro,longitude,ofns_desc,pd_desc,perp_race,law_cat_cd,arrest_key,pd_cd,arrest_date,geometry
0,M,40.785423,23,225433,1,1000328,25-44,None,PL 2223501,M,-73.941939,(null),(null),BLACK,F,245139396,579,2022-05-16,POINT (1000328.573 225432.989)
1,M,40.7426636084461,10,209846,3,984791,18-24,None,CPL5700600,M,-73.9980491084798,(null),(null),BLACK,9,245711809,None,2022-05-26,POINT (984790.599 209849.010)
2,M,40.728981,6,204863,0,984024,25-44,344,PL 1200001,M,-74.000813,ASSAULT 3 & RELATED OFFENSES,ASSAULT 3,BLACK,M,238548378,101,2022-01-02,POINT (984024.669 204864.000)
3,M,40.75393917,14,213954,0,985679,45-64,109,PL 1553005,M,-73.99484365,GRAND LARCENY,"LARCENY,GRAND FROM PERSON,UNCL",BLACK,F,238821779,419,2022-01-08,POINT (985678.600 213957.096)
4,M,40.81591307653,32,236538,1,999433,45-64,236,PL 2650101,M,-73.9451493066481,DANGEROUS WEAPONS,"WEAPONS, POSSESSION, ETC",BLACK,M,239548215,782,2022-01-23,POINT (999432.601 236541.016)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22992,F,40.81591307653,32,236538,0,999433,25-44,106,PL 1200502,M,-73.9451493066481,FELONY ASSAULT,"ASSAULT 2,1,UNCLASSIFIED",BLACK,F,246470343,109,2022-06-11,POINT (999432.601 236541.016)
22993,M,40.8009303727402,25,231080,1,1000558,25-44,343,PL 16515M9,M,-73.9410982410066,OTHER OFFENSES RELATED TO THEF,"THEFT OF SERVICES, UNCLASSIFIE",BLACK,M,245001461,478,2022-05-13,POINT (1000557.601 231083.015)
22994,F,40.737074,13,207813,0,988971,<18,344,PL 1211100,M,-73.982962,ASSAULT 3 & RELATED OFFENSES,OBSTR BREATH/CIRCUL,BLACK,M,247000773,114,2022-06-22,POINT (988971.684 207812.992)
22995,F,40.800896,25,231070,0,1000581,45-64,105,PL 160102A,M,-73.941011,ROBBERY,"ROBBERY,OPEN AREA UNCLASSIFIED",WHITE HISPANIC,F,245240736,397,2022-05-18,POINT (1000581.763 231070.508)


In [56]:
url_params ={"$$app_token": app_token, 
            "$where":"(boro_nm='MANHATTAN')",
            "$limit": "10000000"}


calls = geopandas.read_file(
    requests.get("https://data.cityofnewyork.us/resource/n2zq-pubd.geojson", 
    params=url_params).text).to_crs(epsg=2263)

calls

,disp_ts,closng_ts,latitude,cip_jobs,arrivd_ts,radio_code,incident_time,nypd_pct_cd,longitude,geo_cd_y,incident_date,create_date,patrl_boro_nm,typ_desc,boro_nm,geo_cd_x,add_ts,cad_evnt_id,geometry
0,2022-01-01 00:03:41,2022-01-01 00:58:08,40.765912797000055,Non CIP,NaT,54R9,23:42:04,18,-73.980944143999977,218317,2021-12-31,2022-01-01,PATROL BORO MAN SOUTH,AMBULANCE CASE: RESCUE/TRANSIT,MANHATTAN,989529,2022-01-01 00:02:46,82284236,None
1,2022-01-01 00:12:29,2022-01-01 00:32:48,40.745032053000045,Non CIP,NaT,52D2,00:00:57,10,-74.008190020999962,210709,2022-01-01,2022-01-01,PATROL BORO MAN SOUTH,DISPUTE: OUTSIDE,MANHATTAN,981981,2022-01-01 00:00:57,82284494,None
2,2022-01-01 00:01:42,2022-01-01 07:59:43,40.757665540000062,Non Critical,NaT,39H2,00:01:19,14,-73.985776114999965,215312,2022-01-01,2022-01-01,PATROL BORO MAN SOUTH,OTHER CRIMES (IN PROGRESS): HARASSMENT/OUTSIDE,MANHATTAN,988191,2022-01-01 00:01:19,82284498,None
3,2022-01-01 00:08:55,2022-01-01 00:33:29,40.765003007000075,Non CIP,2022-01-01 00:29:19,11C4,00:01:22,19,-73.964071390999948,217987,2022-01-01,2022-01-01,PATROL BORO MAN NORTH,ALARMS: COMMERCIAL/BURGLARY,MANHATTAN,994203,2022-01-01 00:01:29,82284499,None
4,2022-01-01 00:02:06,2022-01-01 00:14:37,40.791157380000072,Non CIP,2022-01-01 00:02:06,68Q1,00:02:06,24,-73.973349386999985,227515,2022-01-01,2022-01-01,PATROL BORO MAN NORTH,SEE COMPLAINANT: OTHER/INSIDE,MANHATTAN,991630,2022-01-01 00:02:06,82284512,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013085,2022-06-30 23:57:59,2022-07-01 00:20:31,40.779310230000078,Non CIP,2022-06-30 23:57:59,75S,23:57:59,20,-73.983269243999985,223198,2022-06-30,2022-06-30,PATROL BORO MAN NORTH,STATION INSPECTION BY TRANSIT BUREAU PERSONNEL,MANHATTAN,988884,2022-06-30 23:57:59,86689799,None
1013086,2022-06-30 23:58:24,2022-07-01 00:06:16,40.767606430000058,Non CIP,2022-06-30 23:58:24,75M,23:58:24,18,-73.981734273999962,218934,2022-06-30,2022-06-30,PATROL BORO MAN SOUTH,TRAIN RUN/MOBILE ORDER MAINTENANCE SWEEP,MANHATTAN,989310,2022-06-30 23:58:24,86689805,None
1013087,2022-06-30 23:58:33,2022-07-01 00:04:27,40.837510383000051,Non CIP,2022-06-30 23:58:33,75I,23:58:33,33,-73.943360686999938,244407,2022-06-30,2022-06-30,PATROL BORO MAN NORTH,VISIBILITY PATROL: INTERIOR,MANHATTAN,999923,2022-06-30 23:58:33,86689808,None
1013088,2022-06-30 23:58:55,2022-07-01 00:04:47,40.829844745000059,Non CIP,2022-06-30 23:58:55,75S,23:58:55,32,-73.938669672999936,241615,2022-06-30,2022-06-30,PATROL BORO MAN NORTH,STATION INSPECTION BY TRANSIT BUREAU PERSONNEL,MANHATTAN,1001223,2022-06-30 23:58:55,86689817,None


In [62]:
def map_ofns_dec(OFNS_DESC: str) -> str:
    if OFNS_DESC.upper() == "MURDER & NON-NEGL. MANSLAUGHTE":
        type = "VIOLENCE"
    if OFNS_DESC.upper() == "GRAND LARCENY OF MOTOR VEHICLE":
        type = "OTHER"
    elif "LARCENY" in OFNS_DESC.upper():
        type = "LARCENY"
    elif "MURDER" in OFNS_DESC.upper():
        type = "VIOLENCE"
    elif "MANSLAUGHTER" in OFNS_DESC.upper():
        type = "VIOLENCE"
    elif "ASSAULT" in OFNS_DESC.upper():
        type = "VIOLENCE"
    elif "ROBBERY" in OFNS_DESC.upper():
        type = "ROBBERY"
    elif "BURGLARY" in OFNS_DESC.upper():
        type = "BURGLARY"
    elif "DRUGS" in OFNS_DESC.upper():
        type = "DRUGS"
    else:
        type = "OTHER"

    return type

incidents["ofns_type"] = incidents.ofns_desc.apply(map_ofns_dec)
arrests["ofns_type"] = incidents.ofns_desc.apply(map_ofns_dec)

In [76]:
incdients_freq = pandas.DataFrame(incidents.groupby("ofns_type")["cmplnt_num"].count().sort_values(ascending=False))
incdients_freq['prop'] = incidents.groupby("ofns_type")["cmplnt_num"].count()/incdients_freq.sum(axis=0)['cmplnt_num']
incdients_freq

,cmplnt_num,prop
ofns_type,,
LARCENY,26442,0.404578
OTHER,24478,0.374528
VIOLENCE,8490,0.129902
BURGLARY,2406,0.036813
ROBBERY,2007,0.030708
DRUGS,1534,0.023471


In [79]:
arrests_freq = pandas.DataFrame(arrests.groupby("ofns_type")["arrest_key"].count().sort_values(ascending=False))
arrests_freq['prop'] = arrests.groupby("ofns_type")["arrest_key"].count()/arrests_freq.sum(axis=0)['arrest_key']

22997

In [5]:
len(streets)

9347

In [ ]:
test_df = geopandas.sjoin_nearest(streets, incidents, max_distance=5, distance_col="dist")

test_df.groupby(['physicalid']).count()["cmplnt_num"].min()